In [ ]:
# 🚀 Mistral 7B Fairness Training

**Fast, reliable, and excellent performance for fairness!**

## 🎯 **Why Mistral 7B?**
- **Fast Inference**: 2-3x faster than 24B models
- **Reliable Training**: Stable on T4 GPU
- **Excellent Performance**: Much better than GPT-2
- **Easy Deployment**: Perfect for FastAPI servers
- **No Authentication**: No HF token required

## 📋 **Simple Process:**
- **Cell 1**: Setup & Install packages
- **Cell 2**: Upload training data
- **Cell 3**: Load Mistral 7B model
- **Cell 4**: Prepare data & setup training
- **Cell 5**: Train the model
- **Cell 6**: Test & download

**Total time: ~15-20 minutes**


In [ ]:
# 🔧 Cell 1: Setup & Install Packages
print("🔧 Setting up environment for Mistral 7B training...")

# Install packages
!pip install -q torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers>=4.36.0 datasets>=2.14.0 peft>=0.7.0 accelerate>=0.24.0 bitsandbytes>=0.41.0
!pip install -q huggingface_hub

# Import packages
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, TaskType
from datasets import Dataset
from google.colab import files
from huggingface_hub import login, notebook_login
import json
import shutil

print("✅ Packages installed and imported!")

# 🔐 Hugging Face Login
print("🔐 Logging into Hugging Face...")
print("📋 This is required to access the Mistral model")
print("ℹ️  Get your token from: https://huggingface.co/settings/tokens")

try:
    # Use notebook_login for interactive login in Colab
    notebook_login()
    print("✅ Successfully logged into Hugging Face!")
except Exception as e:
    print(f"❌ Login failed: {e}")
    print("💡 Please:")
    print("   1. Go to https://huggingface.co/settings/tokens")
    print("   2. Create a token (Read access is enough)")
    print("   3. Run this cell again")
    raise

# Check GPU
if torch.cuda.is_available():
    print(f"🔥 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ No GPU detected - training will be slower")

print("🚀 Ready for Mistral 7B training!")


In [ ]:
# 📤 Cell 2: Upload Training Data
print("📤 Upload your fairness_politeness_training_v2.jsonl file:")

uploaded = files.upload()

if 'fairness_politeness_training_v2.jsonl' not in uploaded:
    print("❌ Please upload the training file and run this cell again")
else:
    # Preview the data
    with open('fairness_politeness_training_v2.jsonl', 'r') as f:
        lines = f.readlines()
        print(f"✅ Training data uploaded! Dataset contains {len(lines)} examples")
        
        # Show first example
        first_example = json.loads(lines[0])
        print(f"\n🔍 First example preview:\n{first_example['text'][:200]}...")


In [ ]:
# 🤖 Cell 3: Load Mistral 7B Model
print("🤖 Loading Mistral 7B model...")

model_name = "mistralai/Mistral-7B-Instruct-v0.2"
print(f"📋 Model: {model_name}")
print("🎯 Fast, reliable, and excellent for fairness!")

# Load tokenizer
print("🔄 Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print("✅ Tokenizer loaded!")

# Load model with 4-bit quantization
print("🔄 Loading model with 4-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

print("✅ Mistral 7B loaded successfully!")
print(f"📊 Model size: 7B parameters")
print(f"🔥 Device: {next(model.parameters()).device}")
print(f"⚡ Optimized for fast inference!")


In [ ]:
# 📋 Cell 4: Prepare Data & Setup Training
print("📋 Preparing training data and LoRA setup...")

# Format data for Mistral 7B
def format_data_for_mistral(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            item = json.loads(line.strip())
            text = item['text']
            if 'User:' in text and 'Assistant:' in text:
                user_part = text.split('Assistant:')[0].replace('User:', '').strip()
                assistant_part = text.split('Assistant:')[1].strip()
                formatted = f"[INST]{user_part}[/INST]{assistant_part}</s>"
                data.append({"text": formatted})
    return data

# Format and tokenize data
train_data = format_data_for_mistral('fairness_politeness_training_v2.jsonl')
dataset = Dataset.from_list(train_data)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=False, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)

print(f"✅ Formatted {len(train_data)} examples")
print(f"🔍 Example: {train_data[0]['text'][:150]}...")

# Setup LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
)

model = get_peft_model(model, lora_config)

# Print trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"✅ LoRA applied!")
print(f"📊 Trainable: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)")

# Training setup
training_args = TrainingArguments(
    output_dir="./mistral_7b_fairness",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=5,
    save_strategy="epoch",
    warmup_steps=10,
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

print("✅ Training setup complete!")
print(f"📊 Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")


In [ ]:
# 🚀 Cell 5: Train the Model
print("🚀 Starting Mistral 7B fairness training...")
print("⏱️ Expected time: ~15-20 minutes")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# Start training
print("🔥 Training started!")
trainer.train()

print("✅ Training completed!")
print("🎯 Mistral 7B is now trained for fairness!")


In [ ]:
# 🧪 Cell 6: Test & Download Model
print("🧪 Testing the trained Mistral 7B model...")

# Test the model
def test_model(prompt):
    formatted_prompt = f"[INST]{prompt}[/INST]"
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("[/INST]")[-1].strip()

# Test with fairness scenarios
test_prompts = [
    "How should I handle a situation where someone is being discriminated against?",
    "What's the best way to promote equality in the workplace?",
    "How can I be more inclusive in my daily interactions?"
]

print("🔍 Testing fairness responses:")
for i, prompt in enumerate(test_prompts, 1):
    print(f"\n--- Test {i} ---")
    print(f"User: {prompt}")
    response = test_model(prompt)
    print(f"Assistant: {response}")

# Save and download the model
print("\n💾 Saving trained model...")
model.save_pretrained("./mistral_7b_fairness_final")
tokenizer.save_pretrained("./mistral_7b_fairness_final")

# Create a zip file for download
print("📦 Creating download package...")
shutil.make_archive("mistral_7b_fairness_model", 'zip', "./mistral_7b_fairness_final")

print("⬇️ Downloading trained model...")
files.download("mistral_7b_fairness_model.zip")

print("✅ All done! Your Mistral 7B fairness model is ready!")
print("🚀 Fast inference, excellent fairness, perfect for your chatbot!")
